# Data Loading

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [6]:
df_train = pd.read_csv("../data/banking/train.csv", sep=';')
df_test = pd.read_csv("../data/banking/test.csv", sep=';')

In [7]:
# First 10 rows of the training dataset
df_train.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [8]:
df_train.job.unique()

array(['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'], dtype=object)

In [9]:
df_train.marital.unique()

array(['married', 'single', 'divorced'], dtype=object)

In [10]:
df_train.education.unique()

array(['tertiary', 'secondary', 'unknown', 'primary'], dtype=object)

In [11]:
df_train.contact.unique()

array(['unknown', 'cellular', 'telephone'], dtype=object)

In [12]:
df_train.month.unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'jan', 'feb',
       'mar', 'apr', 'sep'], dtype=object)

In [13]:
df_train.poutcome.unique()

array(['unknown', 'failure', 'other', 'success'], dtype=object)

Nos exemplos a seguir, vamos usar a biblioteca [Category Encoders](https://contrib.scikit-learn.org/category_encoders/).

In [14]:
#pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 2.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


## Dummy Encoder

O Dummy Encoder é uma técnica de pré-processamento de dados utilizada para lidar com variáveis categóricas. Ele converte variáveis categóricas em variáveis dummy (também conhecidas como variáveis indicadoras ou variáveis binárias), que são variáveis fictícias que assumem valores de 0 ou 1 para indicar a presença ou ausência de uma determinada categoria.

Passos básicos de como o Dummy Encoder funciona:

1. Identificação de Variáveis Categóricas: O primeiro passo é identificar as variáveis categóricas no conjunto de dados. Essas são variáveis que representam diferentes categorias, mas não têm uma ordem intrínseca.

2. Criação de Variáveis Dummy: Para cada categoria única na variável categórica, o Dummy Encoder cria uma nova variável binária (dummy). Se a observação pertence a uma determinada categoria, a variável dummy correspondente recebe o valor 1; caso contrário, recebe o valor 0.

3. Eliminação de uma Categoria de Referência (opcional): Em muitos casos, é desejável eliminar uma das variáveis dummies para evitar a chamada "armadilha da multicolinearidade", que ocorre quando as variáveis são altamente correlacionadas. Isso é frequentemente feito eliminando uma das categorias, considerando-a como a referência. Todas as outras variáveis dummy são, então, interpretadas em relação a esta referência.

In [15]:
import pandas as pd

df = df_train.copy()
X_train = df['contact']
y_train = df['y']


# Aplicando o Dummy Encoder
df_encoded = pd.get_dummies(X_train, columns=['contact'], drop_first=True)

print(df_encoded)


       telephone  unknown
0          False     True
1          False     True
2          False     True
3          False     True
4          False     True
...          ...      ...
45206      False    False
45207      False    False
45208      False    False
45209       True    False
45210      False    False

[45211 rows x 2 columns]


## Ordinal Encoder

O OrdinalEncoder é uma técnica de pré-processamento de dados usada para codificar variáveis categóricas ordinais em valores numéricos. Variáveis categóricas ordinais têm um significado intrínseco de ordem, ou seja, há uma relação de ordem entre as categorias, mas a distância entre elas não é definida.

Passos da transformação:

1. Atribuição de Rótulos Numéricos: O OrdinalEncoder mapeia cada categoria única da variável categórica para um valor numérico com base na ordem especificada. Esse mapeamento é geralmente definido pelo usuário ou inferido a partir da ordem original das categorias.

2. Codificação da Variável: Substitui as categorias no conjunto de dados pelos rótulos numéricos atribuídos a cada categoria.

Segue um exemplo simples usando Python e scikit-learn:

In [16]:
from sklearn.preprocessing import OrdinalEncoder

df = df_train.copy()
X_train = df[['education']]
y_train = df['y']

# Definindo a ordem das categorias
ordem_categorias = [ 'unknown', 'primary', 'secondary', 'tertiary']

# Inicializando o OrdinalEncoder com a ordem especificada
encoder = OrdinalEncoder(categories=[ordem_categorias])

print(X_train)

# Ajustando e transformando os dados
X_train['education_encoded'] = encoder.fit_transform(X_train[['education']])

print(X_train)


       education
0       tertiary
1      secondary
2      secondary
3        unknown
4        unknown
...          ...
45206   tertiary
45207    primary
45208  secondary
45209  secondary
45210  secondary

[45211 rows x 1 columns]
       education  education_encoded
0       tertiary                3.0
1      secondary                2.0
2      secondary                2.0
3        unknown                0.0
4        unknown                0.0
...          ...                ...
45206   tertiary                3.0
45207    primary                1.0
45208  secondary                2.0
45209  secondary                2.0
45210  secondary                2.0

[45211 rows x 2 columns]


/var/folders/60/xr_t5m4x6lz0yc2gf1r46wtc0000gp/T/ipykernel_60162/2197705283.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['education_encoded'] = encoder.fit_transform(X_train[['education']])


## Target Encoder

O `TargetEncoder` é uma técnica de codificação de variáveis categóricas que utiliza a informação da variável alvo para atribuir valores numéricos às categorias. O processo básico do `TargetEncoder` envolve o seguinte:

1. **Calcula as médias da variável alvo por categoria:** Para cada categoria na variável categórica que você está codificando, o `TargetEncoder` calcula a média da variável alvo (o que geralmente é uma variável binária indicando a classe, por exemplo, 0 ou 1).

2. **Atribui a média da variável alvo à categoria:** O valor numérico atribuído a cada categoria é a média da variável alvo para essa categoria. Isso significa que as categorias que têm uma média alta da variável alvo receberão um valor mais alto, enquanto aquelas com uma média baixa receberão um valor mais baixo.

3. **Codifica as categorias no conjunto de dados:** Substitui as categorias originais no conjunto de dados pelos valores calculados na etapa anterior.

A ideia por trás do `TargetEncoder` é capturar a relação entre a variável categórica e a variável alvo, tornando-a útil em modelos preditivos. No entanto, é importante ter cuidado ao usar o `TargetEncoder` para evitar vazamento de informações. O vazamento ocorre quando as estatísticas (como a média da variável alvo) são calculadas usando informações do conjunto de teste, o que pode resultar em uma avaliação otimista do desempenho do modelo.

A biblioteca `category_encoders` em Python oferece uma implementação do `TargetEncoder` que pode ser usada como exemplificado no código anterior. Certifique-se de verificar a documentação da biblioteca para obter detalhes específicos sobre os parâmetros e opções disponíveis.

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder

df = df_train.copy()

# Split the data into features and target
X_train = df['job']
y_train = df['y']

y_train = y_train.replace({'yes': 0, 'no': 1})

# Initialize the TargetEncoder
encoder = TargetEncoder()

# Fit the encoder on the training data and transform both the training and testing data
X_train_encoded = encoder.fit_transform(X_train, y_train)
# X_test_encoded = encoder.transform(X_test)

# Display the encoded data
print("Encoded Training Data:")
print(X_train_encoded)

Encoded Training Data:
            job
0      0.862444
1      0.889430
2      0.917283
3      0.927250
4      0.881944
...         ...
45206  0.889430
45207  0.772085
45208  0.772085
45209  0.927250
45210  0.917283

[45211 rows x 1 columns]
